# **数据预处理类**：DataPreprocessor

## **简介**

#### 数据预处理是机器学习流程中的核心步骤，旨在优化数据的质量和格式，进而提高模型的表现和稳定性。DataPreprocessor 类提供了一系列常用的数据预处理工具，包括数据标准化、归一化、处理缺失值、类别编码、数据集划分，以及应对类别不平衡的采样方法。接下来将详细介绍每个方法的用途和实现。

## **代码概述**

### **导入必要的库**

#### 首先，导入数据预处理所需的库：

In [1]:
import numpy as np
import pandas as pd
from collections import Counter

### **定义 DataPreprocessor 类**

In [2]:
class DataPreprocessor:
    def __init__(self):
        self.means = None
        self.stds = None

#### __init__ 方法初始化 self.means 和 self.stds，用于存储数据的均值和标准差，以便在标准化数据时进行计算。

### **1.标准化方法：standardize**

#### **方法介绍**

#### standardize 方法用于将数据标准化。标准化会将每个特征的均值调整为 0，标准差调整为 1。这在特征值范围差异较大的数据集上非常有用，可以防止某个特征主导模型的训练。

#### **方法实现**

In [3]:
def standardize(self, X):
    self.means = np.mean(X, axis=0)
    self.stds = np.std(X, axis=0)
    return (X - self.means) / self.stds

* **输入**：X（NumPy 数组格式的数据）
* **输出**：标准化后的数据

#### **为什么使用标准化？**

#### 标准化对许多机器学习算法（如梯度下降法、SVM 和 KNN）非常重要，它可以加快模型收敛速度并提高模型性能。

### **2.最小-最大缩放方法：min_max_scaling**


#### **方法介绍**

#### min_max_scaling 方法将数据缩放到 [0, 1] 范围内，适合于需要归一化的数据集，如决策树、K-means 和神经网络。归一化可以让所有特征位于相同的范围内，消除特征之间的尺度差异。

#### **方法实现**

In [ ]:
def min_max_scaling(self, X):
    return (X - np.min(X, axis=0)) / (np.max(X, axis=0) - np.min(X, axis=0))

* **输入**：X（数据集）
* **输出**：归一化后的数据

#### **使用场景**
* **推荐系统**：归一化特征可以防止具有大范围值的特征对模型产生不良影响。
* **图像处理**：将像素值缩放到 [0, 1]，提高模型的学习效率。

### **3.处理缺失值方法：handle_missing_values**

#### **方法介绍**
#### handle_missing_values 方法用于处理数据集中的缺失值。该方法支持两种策略：
* **均值填充**：用每个特征的均值填充缺失值，适合大多数情况下的缺失值处理。
* **删除样本**：删除包含缺失值的样本，适合缺失值较少或无法填充的情况。

#### **方法实现**

In [ ]:
def handle_missing_values(self, X, strategy='mean'):
    if strategy == 'mean':
        for i in range(X.shape[1]):
            col_mean = np.nanmean(X[:, i])
            X[np.isnan(X[:, i]), i] = col_mean
    elif strategy == 'drop':
        X = X[~np.isnan(X).any(axis=1)]
    return X

* **输入**：X（数据集），strategy（处理策略，默认为 mean）
* **输出**：缺失值处理后的数据

* **输出**：使用均值填充的缺失值数据。

#### **使用策略**
* **均值填充**：适用于缺失值分布较少且数据分布较均匀的情况。
* **删除样本**：适用于缺失值较多或数据集本身样本量足够大的情况。

### **4.编码类别特征方法：encode_categorical**

#### **方法介绍**
#### encode_categorical 方法将类别特征转换为数值数据，使用 one-hot 编码方法。这对于无法直接输入模型的类别特征（如文本标签）非常重要。

#### **方法实现**

In [ ]:
def encode_categorical(self, X, categorical_features):
    for feature in categorical_features:
        unique_vals = np.unique(X[:, feature])
        for val in unique_vals:
            X = np.hstack((X, (X[:, feature] == val).astype(int).reshape(-1, 1)))
        X = np.delete(X, feature, axis=1)
    return X

* **输入**：X（数据集），categorical_features（类别特征索引列表）
* **输出**：编码后的数据

* **输出**：每个类别特征都被转换为一个独立的数值特征。

#### **适用场景**
* **文本分类**：将类别型文本标签转换为数值，以便输入到机器学习模型中。
* **推荐系统**：将用户或物品的类别特征编码为模型可理解的格式。

### **5.训练-测试集划分方法：train_test_split**

#### **方法介绍**
#### train_test_split 方法用于将数据集划分为训练集和测试集，以便评估模型的性能。通过控制 test_size 参数，可以调整测试集的大小。

#### **方法实现**

In [ ]:
def train_test_split(self, X, y, test_size=0.2, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    split_index = int(len(X) * (1 - test_size))
    train_indices, test_indices = indices[:split_index], indices[split_index:]

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

* **输入**：X（特征矩阵），y（标签向量），test_size（测试集比例），random_state（随机种子）
* **输出**：训练集和测试集

#### **重要性**
#### 划分训练集和测试集有助于评估模型的泛化能力，避免过拟合。

### **6.数据不平衡处理方法**

#### **上采样方法**：oversample
#### 此方法用于增加少数类样本数量，使所有类别的样本数量相等，从而缓解类别不平衡问题。

In [ ]:
def oversample(self, X, y):
    counter = Counter(y)
    max_count = max(counter.values())
    X_resampled = []
    y_resampled = []
    for class_label in counter:
        X_class = X[y == class_label]
        y_class = y[y == class_label]
        n_samples = max_count - len(y_class)
        X_resampled.append(X_class)
        y_resampled.append(y_class)
        for _ in range(n_samples):
            idx = np.random.randint(0, len(X_class))
            X_resampled.append(X_class[idx].reshape(1, -1))
            y_resampled.append(class_label)
    return np.vstack(X_resampled), np.concatenate(y_resampled)

#### **下采样方法**：undersample
#### 此方法用于减少多数类样本数量，使所有类别的样本数量相等，从而缓解类别不平衡问题。

In [ ]:
def undersample(self, X, y):
    counter = Counter(y)
    min_count = min(counter.values())
    X_resampled = []
    y_resampled = []
    for class_label in counter:
        X_class = X[y == class_label]
        y_class = y[y == class_label]
        indices = np.random.choice(len(X_class), min_count, replace=False)
        X_resampled.append(X_class[indices])
        y_resampled.append(y_class[indices])
    return np.vstack(X_resampled), np.concatenate(y_resampled)

#### **使用场景**
* **上采样**：适用于少数类样本不足且希望保留所有数据的信息时。
* **下采样**：适用于多数类样本过多且希望简化数据集时。

### **7.数据加载和保存方法**

#### **加载数据方法：load_data**
#### 此方法从 CSV 文件中加载数据，并返回为 NumPy 数组格式。

In [16]:
def load_data(self, filepath):
    return pd.read_csv(filepath).values

* **输入**：文件路径 filepath
* **输出**：数据集（NumPy 数组）

#### **保存数据方法：save_data**
#### 此方法将特征和目标数据保存到 CSV 文件中，便于后续使用。

In [ ]:
def save_data(self, X, y, filepath):
    df = pd.DataFrame(X)
    df['target'] = y
    df.to_csv(filepath, index=False)

* **输入**：X（特征），y（目标标签），filepath（文件路径）
* **输出**：保存的数据集

## **使用示例**
#### **示例一：Iris 数据集（鸢尾花数据集）**
Iris 数据集是经典的分类任务数据集，包含150个样本，每个样本有4个特征，分为三个类别。

In [ ]:
from DataPart.DataPreprocessor import DataPreprocessor
from collections import Counter
from sklearn.datasets import load_iris
# 加载 Iris 数据集
iris = load_iris()
X = iris.data
y = iris.target
# 初始化 DataPreprocessor
preprocessor = DataPreprocessor()
# 处理缺失值
X = preprocessor.handle_missing_values(X, strategy='mean')
# 标准化特征
X_standardized = preprocessor.standardize(X)
# 分割训练集和测试集
X_train, X_test, y_train, y_test = preprocessor.train_test_split(X_standardized, y, test_size=0.2, random_state=42)
# 查看类别分布
print("原始训练集类别分布:", Counter(y_train))
# 使用 SMOTE 进行过采样
X_resampled, y_resampled = preprocessor.oversample(X_train, y_train)
print("过采样后的训练集类别分布:", Counter(y_resampled))

#### **示例二：Wine 数据集（葡萄酒数据集）**
Wine 数据集用于分类任务，根据13个特征将葡萄酒分类为三个类别。

In [ ]:
from DataPart.DataPreprocessor import DataPreprocessor
from collections import Counter
from sklearn.datasets import load_wine
# 加载 Wine 数据集
wine = load_wine()
X = wine.data
y = wine.target
# 初始化 DataPreprocessor
preprocessor = DataPreprocessor()
# 处理缺失值
X = preprocessor.handle_missing_values(X, strategy='mean')
# 标准化特征
X_standardized = preprocessor.standardize(X)
# 分割训练集和测试集
X_train, X_test, y_train, y_test = preprocessor.train_test_split(X_standardized, y, test_size=0.2, random_state=42)
# 查看类别分布
print("原始训练集类别分布:", Counter(y_train))
# 使用欠采样平衡类别
X_resampled, y_resampled = preprocessor.undersample(X_train, y_train)
print("欠采样后的训练集类别分布:", Counter(y_resampled))

#### **示例三：Breast Cancer 数据集（乳腺癌数据集）**
Breast Cancer 数据集用于二分类任务，预测是否存在癌症，基于30个特征。

In [ ]:
from DataPart.DataPreprocessor import DataPreprocessor
from collections import Counter
from sklearn.datasets import load_breast_cancer
# 加载乳腺癌数据集
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
# 初始化 DataPreprocessor
preprocessor = DataPreprocessor()
# 处理缺失值
X = preprocessor.handle_missing_values(X, strategy='mean')
# 标准化特征
X_standardized = preprocessor.standardize(X)
# 分割训练集和测试集
X_train, X_test, y_train, y_test = preprocessor.train_test_split(X_standardized, y, test_size=0.2, random_state=42)
# 查看类别分布
print("原始训练集类别分布:", Counter(y_train))
# 使用 SMOTE 进行过采样
X_resampled, y_resampled = preprocessor.oversample(X_train, y_train)
print("过采样后的训练集类别分布:", Counter(y_resampled))

## **总结**
#### DataPreprocessor 类提供了一套全面的工具集，用于数据预处理和优化。这些方法包括标准化、缩放、处理缺失值、编码类别特征、数据划分和应对类别不平衡问题。通过这些工具，可以更高效地为机器学习模型准备数据，并提高模型的表现和稳定性。

___

# **数据集加载类：** DatasetLoader
## **概述**
DatasetLoader 类是一个用于加载多种常见机器学习数据集的工具。该类封装了多个数据集的下载、解压和预处理功能，简化了数据集的获取和使用过程。支持的数据集包括波士顿房价、手写数字、鸢尾花、乳腺癌、新闻组、LFW人脸、糖尿病、Linnerrud、葡萄酒、奥利维蒂人脸、森林覆盖、RCV1、KDD Cup、加利福尼亚房屋、泰坦尼克号以及CIFAR-10等。

## **功能描述**
- **自动下载和解压：** 对于需要从网络下载并解压的数据集，DatasetLoader 提供了自动化处理。
- **数据预处理：** 加载数据后，自动分离特征（X）和标签（y），并进行必要的编码和转换。
- **统一接口：** 通过统一的 load 方法，根据数据集名称加载对应的数据，简化了数据集的调用方式。
- **扩展性强：** 可以方便地添加新的数据集加载方法，满足不同项目的需求。

## **类结构**
### 类：DatasetLoader
**属性**
- datasets (dict): 存储支持的数据集名称与对应加载方法的映射。

**方法**
- __init__(self): 初始化 datasets 字典，映射数据集名称到加载方法。
- load(self, dataset_name): 根据提供的数据集名称加载对应的数据集。
- 各数据集加载方法（如 load_boston, load_digits, load_iris 等）：具体实现各个数据集的下载、解压和预处理。
- _download_and_extract(self, url): 辅助方法，用于下载和解压缩文件。

## **数据集详情**
以下是 DatasetLoader 支持的主要数据集及其来源：
1. **波士顿房价 (boston)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data)
- 描述: 包含波士顿地区房屋的各种特征及其价格。

2. **手写数字 (digits)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/00360/digits.csv)
- 描述: 包含手写数字图像的特征和标签。

3. **鸢尾花 (iris)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data)
- 描述: 包含鸢尾花的四个特征及其种类。

4. **乳腺癌 (breast_cancer)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/data.csv)
- 描述: 包含乳腺癌患者的诊断特征。

5. **新闻组 (newsgroups)**
- 来源: [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz)
- 描述: 包含20个不同新闻组的文本数据。

6. **LFW人脸 (lfw)**
- 来源: [LFW 数据集](http://vis-www.cs.umass.edu/lfw/lfw.tgz)
- 描述: 包含人脸图像，用于人脸识别任务。

7. **糖尿病 (diabetes)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/diabetes/diabetes.data)
- 描述: 包含糖尿病患者的特征及分类标签。

8. **Linnerrud**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/00448/Linnerrud.txt)
- 描述: 包含Linnerrud数据集的特征和目标值。

9. **葡萄酒 (wine)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/wine.data)
- 描述: 包含葡萄酒的化学成分及其质量评分。

10. **奥利维蒂人脸 (olivetti)**
- 来源: [Olivetti Faces](https://cs.nyu.edu/~roweis/data/olivettifaces.gif)
- 描述: 包含奥利维蒂人脸图像，用于人脸识别。

11. **森林覆盖 (forest_cover)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/covtype)
- 描述: 包含森林覆盖类型的地理和环境特征。

12. **RCV1**
- 来源: [RCV1 数据集](https://paperswithcode.com/dataset/rcv1)
- 描述: 包含新闻文章的文本数据，用于多类别分类。

13. **KDD Cup**
- 来源: [KDD Cup 数据集](https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html)
- 描述: 包含网络入侵检测的数据。

14. **加利福尼亚房屋 (california_housing)**
- 来源: [UCI 机器学习库](https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data)
- 描述: 包含加利福尼亚地区房屋的特征及价格。

15. **泰坦尼克号 (titanic)**
- 来源: [Kaggle 泰坦尼克号数据集]()
- 描述: 包含泰坦尼克号乘客的特征及生存标签。

16. **CIFAR-10**
- 来源: [CIFAR-10 数据集](https://tensorflow.google.cn/datasets/catalog/cifar10)
- 描述: 包含10类物体的彩色图像数据，用于图像分类。

## **使用说明**

### **示例代码**

In [ ]:
from DataPart.DatasetLoader import DatasetLoader
# 创建 DatasetLoader 实例
loader = DatasetLoader()
# 加载鸢尾花数据集
X, y = loader.load('iris')
# 查看数据维度
print(f"特征维度: {X.shape}")
print(f"标签维度: {y.shape}")

### 加载支持的数据集
调用 load 方法并传入数据集名称（不区分大小写）：

In [ ]:
X, y = loader.load('boston')  # 加载波士顿房价数据集
X, y = loader.load('digits')  # 加载手写数字数据集
# 以此类推

对于某些数据集（如 newsgroups, lfw, cifar10），返回值为 None 或处理后的数据：

In [ ]:
data = loader.load('newsgroups')  # 返回 None

## **结论**
DatasetLoader 类提供了一个统一且简便的接口，用于加载多种常见的机器学习数据集。通过封装下载、解压和预处理过程，极大地简化了数据集的使用流程。未来可以通过引入更多功能和优化现有方法，进一步提升其实用性和性能。

---

# **模型评估类**：ModelEvaluator

## **简介**
#### 在机器学习模型中，准确地评估模型的性能是非常重要的。ModelEvaluator 类为此提供了一个全面的工具集，包括计算多种评估指标、生成混淆矩阵、绘制混淆矩阵和 ROC 曲线等功能。接下来将详细介绍每个方法的用途、实现原理及其示例。

### **导入必要的库**
#### 首先，从 dataset.ModelEvaluator 模块导入 ModelEvaluator 类，并导入 numpy 和 matplotlib 库：

In [ ]:
# 导入 ModelEvaluator 类
from Model.ModelEvaluator import ModelEvaluator
import numpy as np
import matplotlib.pyplot as plt

### **创建 ModelEvaluator 实例**

In [ ]:
# 创建 ModelEvaluator 实例
evaluator = ModelEvaluator()

## **方法详解**

### **1. 准确率方法：accuracy**
#### accuracy 方法计算模型预测的正确率，即正确预测的样本占总样本的比例。

In [ ]:
def accuracy(self, y_true, y_pred):
    return np.mean(y_true == y_pred)

* **输入**：真实标签 y_true，预测标签 y_pred
* **输出**：准确率（0 到 1 之间的值

### **2.精确率方法：precision**
#### precision 方法计算模型预测的精确率，即正确预测为正类的样本占所有被预测为正类样本的比例。

#### **实现代码**

In [ ]:
def precision(self, y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    return tp / (tp + fp) if (tp + fp) > 0 else 0

* **输入**：真实标签 y_true，预测标签 y_pred
* **输出**：精确率（0 到 1 之间的值）

### **3.召回率方法：recall**
#### recall 方法计算模型的召回率，即正确预测为正类的样本占所有真实为正类样本的比例。

#### **实现代码**

In [ ]:
def recall(self, y_true, y_pred):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return tp / (tp + fn) if (tp + fn) > 0 else 0

* **输入**：真实标签 y_true，预测标签 y_pred
* **输出**：召回率（0 到 1 之间的值）

### **4. F1 分数方法：f1_score**
#### f1_score 方法计算模型的 F1 分数，这是精确率和召回率的调和平均值，用于综合评价模型的性能。

#### **实现代码**

In [ ]:
def f1_score(self, y_true, y_pred):
    precision = self.precision(y_true, y_pred)
    recall = self.recall(y_true, y_pred)
    return 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

* **输入**：真实标签 y_true，预测标签 y_pred
* **输出**：F1 分数（0 到 1 之间的值）

### **5.综合评估方法：evaluate**
#### evaluate 方法返回所有主要评估指标，包括准确率、精确率、召回率和 F1 分数。

#### **实现代码**

In [ ]:
def evaluate(self, y_true, y_pred):
    return {
        'accuracy': self.accuracy(y_true, y_pred),
        'precision': self.precision(y_true, y_pred),
        'recall': self.recall(y_true, y_pred),
        'f1_score': self.f1_score(y_true, y_pred)
    }

### **6. 混淆矩阵方法：confusion_matrix**
#### confusion_matrix 方法生成混淆矩阵，表示分类模型的性能。

#### **实现代码**

In [ ]:
def confusion_matrix(self, y_true, y_pred):
    classes = np.unique(y_true)
    cm = np.zeros((len(classes), len(classes)), dtype=int)
    for i in range(len(y_true)):
        cm[int(y_true[i]), int(y_pred[i])] += 1
    return cm

* **输入**：真实标签 y_true，预测标签 y_pred
* **输出**：混淆矩阵（二维数组）

### **7. 绘制混淆矩阵方法：plot_confusion_matrix**
#### plot_confusion_matrix 方法绘制混淆矩阵的可视化图。

#### **实现代码**

In [ ]:
def plot_confusion_matrix(self, y_true, y_pred):
    cm = self.confusion_matrix(y_true, y_pred)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(np.unique(y_true)))
    plt.xticks(tick_marks, np.unique(y_true))
    plt.yticks(tick_marks, np.unique(y_true))
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

### **8. 绘制 ROC 曲线方法：plot_roc_curve**
#### plot_roc_curve 方法绘制 ROC 曲线，用于评估二分类模型的性能。

#### **实现代码**

In [ ]:
def plot_roc_curve(self, y_true, y_scores):
    fpr, tpr, _ = self.roc_curve(y_true, y_scores)
    plt.plot(fpr, tpr, label='ROC curve')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

* **输入**：真实标签 y_true，预测分数 y_scores
* **输出**：ROC 曲线图

## **使用示例**
### **示例 一：使用鸢尾花（Iris）数据集评估逻辑回归模型**

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from Model.ModelEvaluator import ModelEvaluator
# 加载鸢尾花数据集
iris = load_iris()
X = iris.data
y = iris.target
# 仅选择二分类问题（类别 0 和 1）
X = X[y != 2]
y = y[y != 2]
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 训练逻辑回归模型
model = LogisticRegression()
model.fit(X_train, y_train)
# 进行预测
y_pred = model.predict(X_test)
y_scores = model.predict_proba(X_test)[:, 1]
# 创建 ModelEvaluator 实例
evaluator = ModelEvaluator()
# 评估模型
metrics = evaluator.evaluate(y_test, y_pred)
print("准确率:", metrics['accuracy'])
print("精确率:", metrics['precision'])
print("召回率:", metrics['recall'])
print("F1 分数:", metrics['f1_score'])
# 绘制混淆矩阵和ROC曲线
evaluator.plot_confusion_matrix(y_test, y_pred)
evaluator.plot_roc_curve(y_test, y_scores)

### **示例二：使用乳腺癌（Breast Cancer）数据集评估支持向量机（SVM）模型**

In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from Model.ModelEvaluator import ModelEvaluator
# 加载乳腺癌数据集
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 训练支持向量机模型
model = SVC(probability=True)
model.fit(X_train, y_train)
# 进行预测
y_pred = model.predict(X_test)
y_scores = model.predict_proba(X_test)[:, 1]
# 创建 ModelEvaluator 实例
evaluator = ModelEvaluator()
# 评估模型
metrics = evaluator.evaluate(y_test, y_pred)
print("准确率:", metrics['accuracy'])
print("精确率:", metrics['precision'])
print("召回率:", metrics['recall'])
print("F1 分数:", metrics['f1_score'])
# 绘制混淆矩阵和ROC曲线
evaluator.plot_confusion_matrix(y_test, y_pred)
evaluator.plot_roc_curve(y_test, y_scores)

## **总结**
#### ModelEvaluator 类提供了一整套工具，帮助您全面评估机器学习模型的性能。从基本的准确率到更复杂的 ROC 曲线分析，您可以获得丰富的信息以优化和改进模型。通过这个类，您可以轻松实现模型性能的多维度分析。

---

# **模型的保存与加载类**：ModelPersistence
## **概述**
ModelPersistence 类提供了简便的方法用于保存和加载机器学习模型。通过使用 joblib 库，该类能够高效地序列化（保存）和反序列化（加载）模型对象，确保模型在不同环境中的可移植性和可重用性。该类设计为静态方法，方便在无需实例化类的情况下直接调用其功能。
## **功能描述**
- **模型保存：** 将训练好的模型对象序列化并保存到指定的文件中，便于后续的模型部署和共享。
- **模型加载：** 从指定文件中反序列化并加载已保存的模型，方便在不同项目或环境中复用模型。
- **错误处理：** 在保存和加载过程中，捕捉并处理可能出现的异常，提供友好的错误提示，提升使用体验。

## **类结构**
### 类：ModelPersistence
### **方法**
- save_model(model, filename) (静态方法)
    - **描述**：将给定的模型对象保存到指定的文件中。
    - **参数：**
        - model：要保存的机器学习模型对象。
        - filename：保存模型的目标文件路径。
    - **返回值：** 无返回值，成功保存后打印确认信息，失败时打印错误信息。
- load_model(filename) (静态方法)
    - **描述**：从指定的文件中加载已保存的模型对象。
    - **参数：**
        - filename：要加载的模型文件路径。
    - **返回值：** 返回加载的模型对象，加载失败时返回 None 并打印错误信息。

## **使用说明：**
### **示例代码**

In [ ]:
from Model.ModelPersistence import ModelPersistence
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
# 加载示例数据
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 训练模型
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
# 保存模型
ModelPersistence.save_model(model, 'random_forest_model.joblib')
# 加载模型
loaded_model = ModelPersistence.load_model('random_forest_model.joblib')
# 使用加载的模型进行预测
if loaded_model is not None:
    predictions = loaded_model.predict(X_test)
    print(f"预测结果: {predictions}")

### 方法调用示例
**保存模型**

In [ ]:
ModelPersistence.save_model(model, 'model_filename.joblib')

参数说明：
- model：需要保存的机器学习模型实例。
- model_filename.joblib：目标保存文件的路径和名称。

**加载模型**

In [ ]:
loaded_model = ModelPersistence.load_model('model_filename.joblib')

参数说明：
- model_filename.joblib：要加载的模型文件的路径和名称。

返回值：
- 成功加载时返回模型对象，失败时返回 None。

## **结论**
ModelPersistence 类为机器学习模型的保存和加载提供了简洁而高效的解决方案。通过封装 joblib 的功能，该类简化了模型持久化的流程，使得模型的复用和部署更加便捷。未来可以通过扩展其功能和增强其安全性，进一步提升其实用性和适用范围。